In [1]:
pip install imbalanced-learn
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
pip install imbalanced-learn

In [24]:
# 데이터 불러오기
df = pd.read_csv("./dataset/players_and_transf.csv")
df

,name,club,age,position,apps,mins,goals,assists,yel,red,shots,ps%,aerials_won,motm,rating,tackles,interceptions,fouls,offsides_won,clearances,dribbled,blocks,own_goals,key_passes,dribblings,fouled,offsides,dispossed,bad_controls,avg_passes,crosses,long_passes,through_passes,league,season,fee,traded,club_position
0,Arjen Robben,Bayern,26,FW,18(6),1779,16,6,1,-,3.4,80.2,0.4,8,8.18,0.6,0.5,0.9,-,-,0.7,-,-,1.8,4,2.2,0.3,2.1,-,34.3,1.5,2,0.2,Bundesliga,2009/2010,NaN,0,1
1,Arjen Robben,Bayern,26,FW,18(6),1779,16,6,1,-,3.4,80.2,0.4,8,8.18,0.6,0.5,0.9,-,-,0.7,-,-,1.8,4,2.2,0.3,2.1,-,34.3,1.5,2,0.2,Bundesliga,2009/2010,NaN,0,1
2,Stefan Kieling,Leverkusen,26,FW,33,2924,21,5,3,-,3,72.4,2.1,6,7.79,1.2,0.7,2.5,-,0.3,0.5,0.1,-,1.6,1.1,3.1,1.8,2.2,-,32.5,0.2,0.4,0.1,Bundesliga,2009/2010,NaN,0,4
3,Stefan Kieling,Leverkusen,26,FW,33,2924,21,5,3,-,3,72.4,2.1,6,7.79,1.2,0.7,2.5,-,0.3,0.5,0.1,-,1.6,1.1,3.1,1.8,2.2,-,32.5,0.2,0.4,0.1,Bundesliga,2009/2010,NaN,0,4
4,Zvjezdan Misimovic,Wolfsburg,28,FW,31,2768,10,13,7,-,2.4,77.5,0.2,8,7.74,1.9,0.6,1.5,-,0.2,1.2,0.1,-,3.3,1.5,2.1,0.3,3.1,-,51.1,2.5,5.7,0.5,Bundesliga,2009/2010,€8.50m,1,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71579,Ramzi Aya,Salernitana,32,Defender,2,136,-,-,1,-,0.5,76.2,-,-,5.31,0.5,-,0.5,-,2,-,-,-,-,-,-,-,-,-,10.5,-,-,-,Serie A,2021/2022,NaN,0,17
71580,Gianluca Pegolo,Sassuolo,41,GK,1,90,-,-,-,-,-,80,-,-,5.00,-,-,-,-,-,-,-,-,-,-,-,-,-,-,30,-,5,-,Serie A,2021/2022,NaN,0,11
71581,Gianluca Pegolo,Sassuolo,41,GK,1,90,-,-,-,-,-,80,-,-,5.00,-,-,-,-,-,-,-,-,-,-,-,-,-,-,30,-,5,-,Serie A,2021/2022,NaN,0,11
71582,Ionut Radu,Inter,25,GK,1,90,-,-,-,-,-,81,-,-,4.90,-,-,-,-,-,-,-,-,-,-,-,-,-,1,21,-,5,-,Serie A,2021/2022,loan transfer,1,2


In [26]:
# 확인
df.columns

Index(['name', 'club', 'age', 'position', 'apps', 'mins', 'goals', 'assists',
       'yel', 'red', 'shots', 'ps%', 'aerials_won', 'motm', 'rating',
       'tackles', 'interceptions', 'fouls', 'offsides_won', 'clearances',
       'dribbled', 'blocks', 'own_goals', 'key_passes', 'dribblings', 'fouled',
       'offsides', 'dispossed', 'bad_controls', 'avg_passes', 'crosses',
       'long_passes', 'through_passes', 'league', 'season', 'traded',
       'club_position'],
      dtype='object')

In [27]:
# 필요없는 columns 없애기
df = df.drop(columns=['name', 'club', 'season','club_position', 'league', 'fee'])

In [29]:
# 반복되어있어서 홀수만 담기
df = df.iloc[1::2]

,age,position,apps,mins,goals,assists,yel,red,shots,ps%,aerials_won,motm,rating,tackles,interceptions,fouls,offsides_won,clearances,dribbled,blocks,own_goals,key_passes,dribblings,fouled,offsides,dispossed,bad_controls,avg_passes,crosses,long_passes,through_passes,traded
1,26,FW,18(6),1779,16,6,1,-,3.4,80.2,0.4,8,8.18,0.6,0.5,0.9,-,-,0.7,-,-,1.8,4,2.2,0.3,2.1,-,34.3,1.5,2,0.2,0
3,26,FW,33,2924,21,5,3,-,3,72.4,2.1,6,7.79,1.2,0.7,2.5,-,0.3,0.5,0.1,-,1.6,1.1,3.1,1.8,2.2,-,32.5,0.2,0.4,0.1,0
5,28,FW,31,2768,10,13,7,-,2.4,77.5,0.2,8,7.74,1.9,0.6,1.5,-,0.2,1.2,0.1,-,3.3,1.5,2.1,0.3,3.1,-,51.1,2.5,5.7,0.5,1
7,24,FW,33(1),3003,22,7,4,-,4.5,70.7,0.7,6,7.73,0.5,0.1,1.9,-,0.9,0.2,0.1,-,1.1,2,1.7,1.6,2.9,-,19.5,0.2,1.1,0.1,0
9,32,FW,23(3),2130,16,2,4,-,2.6,75.5,0.4,2,7.63,1,0.7,2,-,0.7,0.4,0.2,-,1.3,1.2,1.2,1.4,3.1,-,24.2,0.1,0.8,-,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71575,26,D(R),1(1),85,-,-,-,-,-,85.4,0.5,-,5.86,0.5,0.5,-,-,0.5,1.5,-,-,-,-,-,0.5,-,1,20.5,-,1,-,1
71577,32,GK,1,90,-,-,-,-,-,70.4,1,-,5.70,-,-,-,-,1,-,-,-,-,-,-,-,-,-,27,-,12,-,0
71579,32,Defender,2,136,-,-,1,-,0.5,76.2,-,-,5.31,0.5,-,0.5,-,2,-,-,-,-,-,-,-,-,-,10.5,-,-,-,0
71581,41,GK,1,90,-,-,-,-,-,80,-,-,5.00,-,-,-,-,-,-,-,-,-,-,-,-,-,-,30,-,5,-,0


In [30]:
# 괄호 값 없애기
df['apps'] = df['apps'].str.replace(r'\(.*\)', '', regex=True).str.strip()

/tmp/ipykernel_10915/1109888701.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['apps'] = df['apps'].str.replace(r'\(.*\)', '', regex=True).str.strip()


In [31]:
# -를 0으로
df = df.replace('-', 0)

,age,position,apps,mins,goals,assists,yel,red,shots,ps%,aerials_won,motm,rating,tackles,interceptions,fouls,offsides_won,clearances,dribbled,blocks,own_goals,key_passes,dribblings,fouled,offsides,dispossed,bad_controls,avg_passes,crosses,long_passes,through_passes,traded
1,26,FW,18,1779,16,6,1,0,3.4,80.2,0.4,8,8.18,0.6,0.5,0.9,0,0,0.7,0,0,1.8,4,2.2,0.3,2.1,0,34.3,1.5,2,0.2,0
3,26,FW,33,2924,21,5,3,0,3,72.4,2.1,6,7.79,1.2,0.7,2.5,0,0.3,0.5,0.1,0,1.6,1.1,3.1,1.8,2.2,0,32.5,0.2,0.4,0.1,0
5,28,FW,31,2768,10,13,7,0,2.4,77.5,0.2,8,7.74,1.9,0.6,1.5,0,0.2,1.2,0.1,0,3.3,1.5,2.1,0.3,3.1,0,51.1,2.5,5.7,0.5,1
7,24,FW,33,3003,22,7,4,0,4.5,70.7,0.7,6,7.73,0.5,0.1,1.9,0,0.9,0.2,0.1,0,1.1,2,1.7,1.6,2.9,0,19.5,0.2,1.1,0.1,0
9,32,FW,23,2130,16,2,4,0,2.6,75.5,0.4,2,7.63,1,0.7,2,0,0.7,0.4,0.2,0,1.3,1.2,1.2,1.4,3.1,0,24.2,0.1,0.8,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71575,26,D(R),1,85,0,0,0,0,0,85.4,0.5,0,5.86,0.5,0.5,0,0,0.5,1.5,0,0,0,0,0,0.5,0,1,20.5,0,1,0,1
71577,32,GK,1,90,0,0,0,0,0,70.4,1,0,5.70,0,0,0,0,1,0,0,0,0,0,0,0,0,0,27,0,12,0,0
71579,32,Defender,2,136,0,0,1,0,0.5,76.2,0,0,5.31,0.5,0,0.5,0,2,0,0,0,0,0,0,0,0,0,10.5,0,0,0,0
71581,41,GK,1,90,0,0,0,0,0,80,0,0,5.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30,0,5,0,0


In [35]:
# Nan값 없애기
df.isnull().sum()
df.dropna(inplace=True)

age               0
position          0
apps              0
mins              0
goals             0
assists           0
yel               0
red               0
shots             0
ps%               0
aerials_won       0
motm              0
rating            0
tackles           0
interceptions     0
fouls             0
offsides_won      0
clearances        0
dribbled          0
blocks            0
own_goals         0
key_passes        0
dribblings        0
fouled            0
offsides          0
dispossed         0
bad_controls      0
avg_passes        0
crosses           0
long_passes       0
through_passes    0
traded            0
dtype: int64

In [36]:
#데이터 프레임 확인
df.shape

(35683, 32)

In [37]:
from sklearn.model_selection import train_test_split

#dataset 나누기
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, shuffle=True)

# CSV로 저장
train_df.to_csv('./dataset/train.csv', index=False)
val_df.to_csv('./dataset/val.csv', index=False)
test_df.to_csv('./dataset/test.csv', index=False)

# 확인
print("Train shape:", train_df.shape)
print("Validation shape:", val_df.shape)
print("Test shape:", test_df.shape)

Train shape: (28546, 32)
Validation shape: (3568, 32)
Test shape: (3569, 32)
